In [1]:
import cv2
import face_recognition
import numpy as np
import dlib
import os
import datetime
import json
from tkinter import Tk, Label, Entry, Button, StringVar, font, DISABLED, NORMAL
from PIL import Image, ImageTk

# === Folder Setup ===
BASE_DIR = os.path.join(os.getcwd(), "facedetection")
KNOWN_DIR = os.path.join(BASE_DIR, "known_faces")
SESSIONS_DIR = os.path.join(BASE_DIR, "sessions")
os.makedirs(KNOWN_DIR, exist_ok=True)
os.makedirs(SESSIONS_DIR, exist_ok=True)

# === Load Models ===
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# === Camera Init ===
cap = cv2.VideoCapture(0)

# === State Variables ===
known_encodings = []
known_names = []
metadata = []
session_folder = None
is_recording = False
unknown_count = 0
gaze_off_count = 0
MAX_GAZE_OFF = 5
frame_counter = 0

# === GUI Init ===
root = Tk()
root.configure(bg="black")
root.title("FACE DETECTION SYSTEM")

font_style = font.Font(family="Helvetica", size=12, weight="bold")
entry_font = font.Font(family="Helvetica", size=11, weight="bold")

name_var = StringVar()
status_var = StringVar(value="READY")
unknown_var = StringVar(value="UNKNOWN COUNT: 0")
gaze_var = StringVar(value="GAZE OFF-FOCUS: 0 / 5")

# === Functions ===
def load_known_faces():
    known_encodings.clear()
    known_names.clear()
    for file in os.listdir(KNOWN_DIR):
        path = os.path.join(KNOWN_DIR, file)
        image = face_recognition.load_image_file(path)
        enc = face_recognition.face_encodings(image)
        if enc:
            known_encodings.append(enc[0])
            known_names.append(os.path.splitext(file)[0])
    status_var.set(f"LOADED {len(known_names)} KNOWN FACE(S)")

def save_known_face():
    name = name_var.get().strip().upper()
    if not name:
        status_var.set("ENTER NAME FIRST")
        return
    ret, frame = cap.read()
    if not ret:
        status_var.set("CAMERA ERROR")
        return
    filepath = os.path.join(KNOWN_DIR, f"{name}.jpg")
    cv2.imwrite(filepath, frame)
    img = face_recognition.load_image_file(filepath)
    enc = face_recognition.face_encodings(img)
    if not enc:
        os.remove(filepath)
        status_var.set("FACE NOT CLEAR. TRY AGAIN.")
        return
    known_encodings.append(enc[0])
    known_names.append(name)
    status_var.set(f"SAVED: {name}")

def start_surveillance():
    global session_folder, metadata, is_recording, unknown_count, gaze_off_count, frame_counter
    if not known_names:
        status_var.set("NO KNOWN FACE. SAVE FIRST.")
        return
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")
    session_folder = os.path.join(SESSIONS_DIR, timestamp)
    os.makedirs(session_folder, exist_ok=True)
    metadata.clear()
    unknown_count = 0
    gaze_off_count = 0
    frame_counter = 0
    unknown_var.set("UNKNOWN COUNT: 0")
    gaze_var.set("GAZE OFF-FOCUS: 0 / 5")
    is_recording = True
    resume_btn.config(state=DISABLED)
    stop_btn.config(state=NORMAL)
    status_var.set("RECORDING STARTED")

def stop_surveillance():
    global is_recording
    is_recording = False
    status_var.set("RECORDING STOPPED")
    stop_btn.config(state=DISABLED)

def resume_surveillance():
    global gaze_off_count, is_recording
    gaze_off_count = 0
    gaze_var.set("GAZE OFF-FOCUS: 0 / 5")
    is_recording = True
    resume_btn.config(state=DISABLED)
    stop_btn.config(state=NORMAL)
    status_var.set("RESUMED")

def exit_app():
    if metadata and session_folder:
        with open(os.path.join(session_folder, "metadata.json"), "w") as f:
            json.dump(metadata, f, indent=4)
    cap.release()
    root.destroy()

def is_gaze_off_focus(landmarks):
    left_eye = landmarks[36:42]
    right_eye = landmarks[42:48]
    def eye_direction(eye_points):
        eye_width = np.linalg.norm(eye_points[0] - eye_points[3])
        eye_height = (np.linalg.norm(eye_points[1] - eye_points[5]) + np.linalg.norm(eye_points[2] - eye_points[4])) / 2
        ratio = eye_height / eye_width
        return ratio < 0.2  # eye closed or looking away
    return eye_direction(np.array(left_eye)) or eye_direction(np.array(right_eye))

def process_frame():
    global unknown_count, gaze_off_count, is_recording, frame_counter

    ret, frame = cap.read()
    if not ret:
        root.after(5, process_frame)
        return

    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    dets = face_detector(rgb_small, 1)
    face_locations = []
    face_encodings = []

    for det in dets:
        shape = landmark_predictor(rgb_small, det)
        top, right, bottom, left = det.top()*4, det.right()*4, det.bottom()*4, det.left()*4
        face_locations.append((top, right, bottom, left))

        enc = face_recognition.face_encodings(rgb_small, [(det.top(), det.right(), det.bottom(), det.left())])
        if enc:
            face_encodings.append(np.array(enc[0]))

    for ((top, right, bottom, left), encoding) in zip(face_locations, face_encodings):
        name = "UNKNOWN"
        matches = face_recognition.compare_faces(known_encodings, encoding)
        if True in matches:
            name = known_names[matches.index(True)]

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        if is_recording:
            face_img = frame[top:bottom, left:right]
            gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
            rect = dlib.rectangle(0, 0, right - left, bottom - top)
            shape = landmark_predictor(gray, rect)
            landmarks = [(p.x, p.y) for p in shape.parts()]

            if is_gaze_off_focus(landmarks):
                gaze_off_count += 1
                gaze_var.set(f"GAZE OFF-FOCUS: {gaze_off_count} / {MAX_GAZE_OFF}")
                snapshot_name = f"gaze_violation_{datetime.datetime.now().strftime('%H%M%S')}.jpg"
                cv2.imwrite(os.path.join(session_folder, snapshot_name), frame)
                metadata.append({
                    "filename": snapshot_name,
                    "timestamp": datetime.datetime.now().isoformat(),
                    "event": "gaze_off_focus"
                })
                if gaze_off_count >= MAX_GAZE_OFF:
                    is_recording = False
                    resume_btn.config(state=NORMAL)
                    stop_btn.config(state=DISABLED)
                    status_var.set("PAUSED AFTER 5 GAZE VIOLATIONS.")

        if is_recording and name == "UNKNOWN":
            unknown_count += 1
            unknown_var.set(f"UNKNOWN COUNT: {unknown_count}")
            fname = f"unknown_{datetime.datetime.now().strftime('%H%M%S')}.jpg"
            cv2.imwrite(os.path.join(session_folder, fname), frame)
            metadata.append({
                "filename": fname,
                "timestamp": datetime.datetime.now().isoformat(),
                "face_box": {"top": top, "right": right, "bottom": bottom, "left": left}
            })

    if is_recording:
        frame_counter += 1
        fpath = os.path.join(session_folder, f"frame_{frame_counter:05d}.jpg")
        cv2.imwrite(fpath, frame)

    preview = cv2.resize(frame, (360, 270))
    img_pil = Image.fromarray(cv2.cvtColor(preview, cv2.COLOR_BGR2RGB))
    img_tk = ImageTk.PhotoImage(img_pil)
    preview_label.configure(image=img_tk)
    preview_label.image = img_tk

    root.after(5, process_frame)

# === GUI Layout ===
Label(root, text="NAME:", font=font_style, fg="white", bg="black").grid(row=0, column=0)
Entry(root, textvariable=name_var, font=entry_font, bg="black", fg="white", insertbackground="white").grid(row=0, column=1)

Button(root, text="SAVE FACE", font=font_style, bg="gray", fg="black", command=save_known_face).grid(row=1, column=0)
Button(root, text="START", font=font_style, bg="green", fg="black", command=start_surveillance).grid(row=1, column=1)
stop_btn = Button(root, text="STOP", font=font_style, bg="red", fg="black", command=stop_surveillance, state=DISABLED)
stop_btn.grid(row=1, column=2)
resume_btn = Button(root, text="RESUME", font=font_style, bg="orange", fg="black", command=resume_surveillance, state=DISABLED)
resume_btn.grid(row=1, column=3)
Button(root, text="EXIT", font=font_style, bg="white", fg="black", command=exit_app).grid(row=1, column=4)

Label(root, textvariable=status_var, font=font_style, fg="white", bg="black").grid(row=2, column=0, columnspan=5)
Label(root, textvariable=unknown_var, font=font_style, fg="white", bg="black").grid(row=3, column=0, columnspan=2)
Label(root, textvariable=gaze_var, font=font_style, fg="white", bg="black").grid(row=3, column=2, columnspan=3)

preview_label = Label(root, bg="black")
preview_label.grid(row=4, column=0, columnspan=5)

# === Run App ===
load_known_faces()
process_frame()
root.protocol("WM_DELETE_WINDOW", exit_app)
root.mainloop()
